In [41]:
import pyvista as pv
import meshio
import numpy as np
import psycopg2
import hdf5storage
import h5py
import pickle
import logging
from sys import getsizeof
from dotenv import dotenv_values
import matplotlib.pyplot as plt

pv.global_theme.transparent_background = True


config = dotenv_values("../.env")
# Filepaths for script
initMeshPath = config["currentDirectory"] +"meshes/fullAssemblyContiguous/fullFishContiguous.msh"
tempMeshPath = config["backupDirectory"] + "data/visualizations/holdpls/"
outputPath = config["currentDirectory"] + "data/visualizations/"


In [42]:
# Helper functions for working with database
def get_db_connection():
    conn = psycopg2.connect(
        dbname='simDB',
        user='user',
        password='password',
        host='localhost',
        port='5432'
    )
    return conn

def fetch_trial_data_singleTimestep(conn, trial_id, n=1, start_timestep=0, num_timesteps=500):
    try:
        cur = conn.cursor()
        
        # Fetch data for the given trial_id starting from start_timestep
        query = '''
        SELECT timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat
        FROM simulation_data
        WHERE trial_id = %s AND timestep >= %s AND timestep < %s
        ORDER BY timestep ASC;
        '''
        cur.execute(query, (trial_id, start_timestep, start_timestep + 1))
        rows = cur.fetchall()
        
        # Deserialize data
        data = []
        for row in rows:
            timestep, simulation_time, input_data_bin, output_data_bin, state_data_bin, y_ref_bin, x_hat_bin = row
            input_data = np.array(pickle.loads(input_data_bin))
            output_data = np.array(pickle.loads(output_data_bin))
            state_data = np.array(pickle.loads(state_data_bin))
            y_ref = np.array(pickle.loads(y_ref_bin))
            x_hat = np.array(pickle.loads(x_hat_bin))
            data.append((timestep, simulation_time, input_data, output_data,state_data, y_ref, x_hat))
        
        cur.close()
        return data
    except Exception as e:
        print(f"Error fetching data for trial_id {trial_id}: {e}")
        return None
    


In [43]:
# Trial Parameters
trial_ids = [192]
trial_id = trial_ids[0]
timesteps = 500


In [44]:
### Get offsets for reference trajectory
# Load data from database
conn = get_db_connection()
data = fetch_trial_data_singleTimestep(conn, trial_ids[0], 1, start_timestep=0, num_timesteps=1)
conn.close()
timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
# Get the reference trajectory
refOffsets = np.copy(output_data)


## Plot single frame with dots on centerline

In [58]:
sizes = np.ones((20)) * 10
# Plot first frame of first trial with dots over centerline position
# create plotter objects
plotter = pv.Plotter(shape=(1,1), border=False, window_size=[11000, 1200])

# Load data from database
conn = get_db_connection()
data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=0, num_timesteps=1)
conn.close()
timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]

# Check shape of reference trajectory
uncenteredRefTraj = y_ref + refOffsets
# Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
x_refs = uncenteredRefTraj[::2]
z_refs = uncenteredRefTraj[1::2]
referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
path = referenceCurve
offsetPath = np.copy(path)
offsetPath[:, 1] = offsetPath[:, 1] - 122


# #### Draw Surface for reference trajectory
# # Number of samples (in depth dir) and number of points along centerline 
# nsamples = 3
# ntraces = 20  
# # Define the Z spacing of your 2D section
# z_spacing = 60
# # Create structured points draping down from the path
# points = np.repeat(path, nsamples, axis=0)
# # repeat the Z locations across
# tp = np.arange(0, z_spacing * nsamples, z_spacing)
# # Offset the Z locations 
# tp = path[:, 1][:, None] - tp
# points[:, 1] = tp.ravel()
# # Make a StructuredGrid from the structured points
# grid = pv.StructuredGrid()
# grid.points = points
# grid.dimensions = nsamples, ntraces, 1
# # Add the data array - note the ordering
# grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")

### Plot actual centerline from output_data
# Load data from database
conn = get_db_connection()
data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=0, num_timesteps=1)
conn.close()
timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]

# Check shape of output data
uncenteredOutputTraj = output_data
# Turn output trajectory from 40x1 to 20x2 array by taking every other point
x_outs = uncenteredOutputTraj[::2]
z_outs = uncenteredOutputTraj[1::2]

outputCurve = np.column_stack((x_outs, np.zeros(np.size(x_outs)), np.zeros(z_outs.shape)))#z_outs))
offsetOutputCurve = np.copy(outputCurve)
offsetOutputCurve[:, 1] = offsetOutputCurve[:, 1] - 122

# load mesh from .vtu file in tempMeshPath
mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{0}.vtu")

# Add meshes to top view window
# plotter.subplot(0, 0)
plotter.add_mesh(mesh, show_scalar_bar=False, name='mesh', style="points",point_size=100, show_edges=True, edge_color='black')
# plotter.add_mesh(grid, clim=[-1, 1], show_scalar_bar=False, name='reference', opacity=0.5)
# plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name='refPath', render_points_as_spheres=True, point_size=10)
plotter.add_mesh(pv.PolyData(offsetOutputCurve), color='red', line_width=5, name='outputPath', render_points_as_spheres=True, point_size=200)
# Stylize mesh to be transparent
plotter.camera_position = 'yz'
plotter.camera.roll = 180
plotter.camera.azimuth = 90
plotter.camera.elevation = 90
plotter.enable_parallel_projection()
plotter.camera.zoom(9.15)
# plotter.show()

# Save plot as svg
# plotter.save_graphic(outputPath + "fishWDots.svg")
# Save plot as png
plotter.screenshot(outputPath + "fishWDots.png")



pyvista_ndarray([[[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0]],

                 [[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0]],

                 [[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0]],

                 ...,

                 [[255, 255, 255,   0],
                  [255, 255, 255,   0],
                  [255, 255, 255,   0],
                  ...,
                  [255, 255, 255,   0],